# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [42]:
import json
with open ("/data/negotiated_rates.json") as f:
    data=json.load(f)
def average_rate(file):
    """(file) -> (float)
       This function reads all the permitted amounts attributes from the data using a file as its input.
       The average of the permitted amounts is then determined by adding up all the rates and dividing that sum by the number of rates.
       
       >>> average_rate('/data/negotiated_rates.json')
       38.67
    """
    
    rates=[]
    count=0
    for out_of_network in data['out_of_network']:
        for allowed_amounts in out_of_network['allowed_amounts']:
            count +=1
            rates.append(allowed_amounts['payments']['allowed_amount'])
            total_rates= sum(rates)/(count)
            
    return round(total_rates,2)

In [43]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

In [44]:
import doctest
doctest.run_docstring_examples(average_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    average_rate('/data/negotiated_rates.json')
Expecting:
    38.67
ok


---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [60]:
import json
with open ("/data/negotiated_rates.json") as f:
    data=json.load(f)
def get_rate(file, billing_code, service_code):
    """(file,str,int) ->float
        The JSON file, the billing code, and the service code are the three inputs for this function.
        Each billing code identifies the method offered, and each service code identifies the kind of service offered.
        Combining these yields the permitted amount for these specific billing codes and service codes is given as output.
        It should return none if the combination cannot be found.
        
        >>> get_rate('/data/negotiated_rates.json','G0283','11')
        8.78
        >>> get_rate('/data/negotiated_rates.json','97140','11') 
        20.03
        >>> get_rate('/data/negotiated_rates.json','97110','12')
        26.62
    """
    
    for out_of_network in data['out_of_network']:
        if out_of_network["billing_code"] == billing_code:
            for allowed_amounts in out_of_network['allowed_amounts']:
                if allowed_amounts["service_code"] == service_code:
                    return(allowed_amounts['payments']['allowed_amount'])
                else:
                     None

    

In [61]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

In [50]:
import doctest
doctest.run_docstring_examples(get_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_rate('/data/negotiated_rates.json','G0283','11')
Expecting:
    8.78
ok
Trying:
    get_rate('/data/negotiated_rates.json','97140','11') 
Expecting:
    20.03
ok
Trying:
    get_rate('/data/negotiated_rates.json','97110','12')
Expecting:
    26.62
ok


---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [51]:
from datetime import datetime

def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    """"(file,str,int,int,str) -> float
        The function provides modified rates based on patient age and day of the week using five parameters as inputs: file name, billing code, service code, patient age, and visit date.
        It determines the adjusted rates using the following guidelines:
        If it's Monday and the patient is over 65, 50% of the permitted amount will be charged.
        If the patient is above 65, 50% of the permitted amount is charged.
        On Mondays, 75% of the permitted amount is charged.
        If neither of the conditions is met, the whole sum is charged.
        This function will take up the date in the form of %Y-%m-%d from datetime.strptime 
        and the day of the week is calculated as an integer, where Monday is 0 and Sunday is 6.
        
        >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
        8.78
        >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')
        6.58
    """
     
    for out_of_network in data['out_of_network']:
        if out_of_network["billing_code"] == billing_code:
            for allowed_amounts in out_of_network['allowed_amounts']:
                if allowed_amounts["service_code"] == service_code:
                    day = datetime.strptime(visit_date,"%Y-%m-%d").weekday()
                    if day== 0 and pat_age >= 65:
                        amount_charge= 0.5* allowed_amounts['payments']['allowed_amount']
                        return (round(amount_charge,2))
                    elif pat_age >= 65:
                        amount_charge= 0.5* allowed_amounts['payments']['allowed_amount']
                        return (round(amount_charge,2))
                    elif day== 0:
                        amount_charge= 0.75* allowed_amounts['payments']['allowed_amount']
                        return (round(amount_charge,2))
                    else:
                        return(allowed_amounts['payments']['allowed_amount'])
                        
                
            
            
            

In [52]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

In [53]:
import doctest
doctest.run_docstring_examples(get_adjusted_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
Expecting:
    8.78
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')
Expecting:
    6.58
ok


---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [81]:
import json
import csv
from datetime import datetime

def summarize_reimbursement(visits,negotiated_rates):
    """(file) -> dict
      The function will take inputs from file and compute an adjusted rate.
      The adjusted rate is calculated by matching them to the json parameters i.e billing code,service code,patient age,visit date 
      and returns two dictionaries.
      >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] 
      277.67
      >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03']
      378.72
      """

    rows = []
    with open(visits,"r") as f:
        csv_reader = csv.reader(f)
        csv_header = next(csv_reader)
        for row in csv_reader:
            temp_dict = {}
            for k,v in zip(csv_header, row):
                temp_dict[k.lower()] = v
            rows.append(temp_dict)
    by_month = {}
    by_hospital = {}
    for each_row in rows:
        adjusted_rate = get_adjusted_rate(negotiated_rates, each_row['billing_code'], each_row['service_code'], int(each_row['patient_age']), each_row['visit_date'])
        visit_month = datetime.strptime(each_row['visit_date'], r"%Y-%m-%d").strftime(r"%Y-%m")
        hospital = each_row["hospital"]
        if adjusted_rate != None:
            if visit_month in by_month.keys():
                by_month[visit_month].append(adjusted_rate)
            else:
                by_month[visit_month] = [adjusted_rate]
            if hospital in by_hospital.keys():
                by_hospital[hospital].append(adjusted_rate)
            else:
                by_hospital[hospital] = [adjusted_rate]
    for i in by_month.keys():
        by_month[i] = round(sum(by_month[i]), 2)
    for i in by_hospital.keys():
        by_hospital[i] = round(sum(by_hospital[i]), 2)
    return by_month, by_hospital
            

In [82]:
by_month,by_hospital= summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

In [83]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)

In [84]:
import doctest
doctest.run_docstring_examples(summarize_reimbursement, globals(), verbose=True)

Finding tests in NoName
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] 
Expecting:
    277.67
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03']
Expecting:
    378.72
ok
